In [2]:
from cv2 import aruco, cvtColor, COLOR_BGR2GRAY, LUT, imread
import numpy as np
from glob import glob
import pandas as pd

In [3]:
c_rpi = 4740
x0_rpi = 0.
y0_rpi = 0.
dist_rpi = np.zeros(5)

pixelsize_rpi = 1.4

kameratypen = pd.DataFrame([[0, "rpi", c_rpi, x0_rpi, y0_rpi, dist_rpi[0], dist_rpi[1], dist_rpi[2], dist_rpi[3], dist_rpi[4], pixelsize_rpi]],
                           columns=["kameratyp", "name", "c", "x0", "y0", "d1", "d2", "d3", "d4", "d5", "pixelsize"]).astype({"kameratyp": int}).set_index(["kameratyp"])
kameras = pd.DataFrame([], columns=["kamera", "kameratyp", "name", "dc", "dx0", "dy0"]).astype(
    {"kamera": int, "kameratyp": int, "dc": float, "dx0": float, "dy0": float}).set_index(["kamera"])

pictures = pd.DataFrame(columns=["img", "kamera",
                        "pfad", "t1", "t2", "t3", "r1", "r2", "r3"],  dtype=float).astype({"img": int, "kamera": int, "pfad": str}).set_index(["img"])

In [4]:
pfad = "../../bilderserien/PasspunktPos/8a25d87e-9f2a-48da-872d-36ce0d6b66d9.jpg/*.jpg"
bilderpfade = glob(pfad)

rpi_kameras = list(set([i[-12:-4] for i in bilderpfade]))
rpi_kameras.sort()

for i in rpi_kameras:
    kamera_id = len(kameras)
    kameras.loc[kamera_id] = [0, i, 0., 0., 0.]

for i in bilderpfade:
    kamera = kameras.index[kameras['name'] == i[-12:-4]][0]
    pictures.loc[len(pictures), ['kamera', 'pfad']] = [int(kamera), i]
pictures = pictures.astype({"kamera": int})

In [5]:
aruco_dict = aruco.extendDictionary(32, 3)
parameter = aruco.DetectorParameters()
parameter.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

In [6]:
corners = []
for j, img in pictures.iterrows():
    imgCV = imread(img['pfad'])
    gray = cvtColor(imgCV, COLOR_BGR2GRAY)
    tmp_corners, tmp_ids, t = aruco.detectMarkers(
        gray, aruco_dict, parameters=parameter)
    x,y,farbtiefe = imgCV.shape
    x /= 2.
    y /= 2.
    pixel = kameratypen.iloc[kameras.iloc[img['kamera']]['kameratyp']]['pixelsize']
    for c, i in zip(tmp_corners, tmp_ids):
        size = np.linalg.norm(c[0][0] - c[0][1])
        # print(size)
        if size < 100:
            continue
        for k in range(len(c[0])):
            corners.append([j, i[0], k, (c[0][k][0]-x)*pixel, (c[0][k][1]-y)*pixel])
corners = pd.DataFrame(corners, columns=["img", "marker", "ecke", "x'", "y'"])

corners.set_index(["img", "marker", "ecke"], inplace=True)

In [7]:
corners

x'           y'
img marker ecke                          
0   13     0    -1538.688205  -257.016211
           1    -1399.847729  -141.943799
           2    -1425.948279   117.290674
           3    -1570.110339    11.732178
    29     0     -478.780945   185.393701
...                      ...          ...
22  9      3     -564.740979 -1955.345667
    10     0     1524.499854 -2050.149524
           1     1293.365186 -2051.190295
           2     1295.849707 -2284.198096
           3     1531.295801 -2283.048633

[1048 rows x 2 columns]

In [8]:
def get_kamera(kameras, kid):
    kamera = kameras.iloc[kid]
    kameratyp = kameratypen.iloc[kamera["kameratyp"]]
    mtx = np.array([[kamera["dc"] + kameratyp["c"], 0, kamera["dx0"] + kameratyp["x0"]],
                    [0, kamera["dc"] + kameratyp["c"],
                        kamera["dy0"] + kameratyp["y0"]],
                    [0, 0, 1]])
    dist = np.array(kameratyp[["d1", "d2", "d3", "d4", "d5"]])
    return mtx.astype(np.float64), dist.astype(np.float64)

In [9]:
coords = pd.DataFrame([[0, 0, -7.8, 7.8, 0.0],
                       [0, 1, 7.8, 7.8, 0.0],
                       [0, 2, 7.8, -7.8, 0.0],
                       [0, 3, -7.8, -7.8, 0.0]], columns=["marker", "ecke", "X", "Y", "Z"]).astype({"marker": int, "ecke": int}).set_index(["marker", "ecke"])

## Erste Kamera orientieren

Luhmann, S.290

Rechenweg nach https://de.wikipedia.org/wiki/Projektionsmatrix_(Computer_Vision)#Berechnung_der_Projektionsmatrix_aus_Punktkorrespondenzen

$$ x' = P \cdot X $$
$$ x' \times PX = 0 $$
$$
	\begin{bmatrix}
		\mathbf{0}^T & -w_i\mathbf{X}_i & y_i\mathbf{X}_i \\
	 w_i\mathbf{X}_i & \mathbf{0}^T & -x_i\mathbf{X}_i \\
	 -y_i\mathbf{X}_i & x_i\mathbf{X}_i & \mathbf{0}^T	
	\end{bmatrix}
	\begin{pmatrix}
		\mathbf{P}^{1T} \\
		\mathbf{P}^{2T} \\
		\mathbf{P}^{3T}
	\end{pmatrix} = 0
$$

In [14]:
X[:3].T
x_strich.T

array([[ 1.17408252e+03, -1.28175759e+03,  1.00000000e+00],
       [ 2.19340386e+03, -1.26623079e+03,  1.00000000e+00],
       [ 2.26310923e+03, -2.96436328e+02,  1.00000000e+00],
       [ 1.09639380e+03, -3.25255957e+02,  1.00000000e+00]])

In [15]:
[[0, -a[2]*b, a[1]*b] for a,b in zip(x_strich.T.ravel, X[:3].T)]

TypeError: 'builtin_function_or_method' object is not iterable

In [18]:
K, dist = get_kamera(kameras, 0)
K

array([[4.74e+03, 0.00e+00, 0.00e+00],
       [0.00e+00, 4.74e+03, 0.00e+00],
       [0.00e+00, 0.00e+00, 1.00e+00]])

In [12]:
verkn = corners.join(coords, on=["marker", "ecke"]).dropna().loc[0].to_numpy()
X = np.c_[verkn[:, 2:5],np.ones(len(verkn))].T
X

array([[-7.8,  7.8,  7.8, -7.8],
       [ 7.8,  7.8, -7.8, -7.8],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 1. ,  1. ,  1. ,  1. ]])

In [13]:
x_strich = np.c_[verkn[:, :2], np.ones(len(verkn))].T
x_strich

array([[ 1.17408252e+03,  2.19340386e+03,  2.26310923e+03,
         1.09639380e+03],
       [-1.28175759e+03, -1.26623079e+03, -2.96436328e+02,
        -3.25255957e+02],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00]])

In [16]:
R = np.eye(3)
X0 = np.array([0, 0, 30])

In [17]:
P = K@np.c_[R, -R@X0]


NameError: name 'K' is not defined

In [ ]:
Test = P@X
Test  /= Test[2]
Test

array([[ 1.2324e+03, -1.2324e+03, -1.2324e+03,  1.2324e+03],
       [-1.2324e+03, -1.2324e+03,  1.2324e+03,  1.2324e+03],
       [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00]])

In [ ]:
np.linalg.svd(P)

NameError: name 'np' is not defined